In [1]:
import pandas as pd

# Load Data
cig_data = pd.read_csv("data/input/CDC_1970-2018.csv")
cpi_data = pd.read_excel("data/input/CPI_1913_2019.xlsx", skiprows=11)

# Clean Tobacco Data
measure_map = {
    "Average Cost per pack": "cost_per_pack",
    "Cigarette Consumption (Pack Sales Per Capita)": "sales_per_capita",
    "Federal and State tax as a Percentage of Retail Price": "tax_percent",
    "Federal and State Tax per pack": "tax_dollar",
    "Gross Cigarette Tax Revenue": "tax_revenue",
    "State Tax per pack": "tax_state"
}

cig_data["measure"] = cig_data["SubMeasureDesc"].map(measure_map)
cig_data = cig_data[["LocationAbbr", "LocationDesc", "Year", "Data_Value", "measure"]]
cig_data.columns = ["state_abb", "state", "Year", "value", "measure"]

final_data = cig_data.pivot(index=["state", "Year"], columns="measure", values="value").reset_index()

# Clean CPI Data
cpi_data = cpi_data.melt(id_vars=["Year"], value_vars=["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"],
                         var_name="month", value_name="index")
cpi_data = cpi_data.groupby("Year", as_index=False).agg({"index": "mean"})

# Merge and Adjust Prices
final_data = final_data.merge(cpi_data, on="Year", how="left")
final_data["price_cpi"] = final_data["cost_per_pack"] * (218 / final_data["index"])

# Save Output
final_data.to_csv("data/output/TaxBurden_Data.txt", sep="\t", index=False)
final_data.to_pickle("data/output/TaxBurden_Data.pkl")


FileNotFoundError: [Errno 2] No such file or directory: 'data/input/CDC_1970-2018.csv'